# Signals Boosting

NOTE: This notebook depends upon the the Retrotech dataset. If you have any issues, please rerun the [Setting up the Retrotech Dataset](1.setting-up-the-retrotech-dataset.ipynb) notebook.

In [1]:
import sys
sys.path.append('..')
from aips import *  
from pyspark.sql import SparkSession
engine = get_engine()
spark = SparkSession.builder.appName("AIPS").getOrCreate()
products_collection = engine.get_collection("products")

## Keyword Search with No Signals Boosting

### Listing 4.5

In [2]:
# %load -s product_search_request solr
def product_search_request(query):
    return {
        "query": query,
        "fields": ["upc", "name", "manufacturer", "score"],
        "limit": 5,
        "params": {
            "qf": "name manufacturer longDescription",
            "defType": "edismax",
            "indent": "true",
            "sort": "score desc, upc asc"
        }
    }


In [3]:
query = "ipod firewire dock cable"
request = product_search_request(query)
response = products_collection.search(request)
display_product_search(query, engine.docs_from_response(response))

{'upc': '885909411955', 'name': 'Apple&#xAE; - Dock Connector-to-USB Cable', 'manufacturer': 'Apple&#xAE;', 'score': 6.5765057}
Apple&#xAE; - Dock Connector-to-USB Cable
{'upc': '722868817728', 'name': "Belkin - 1' FireWire Cable", 'manufacturer': 'Belkin', 'score': 6.5045505}
Belkin - 1' FireWire Cable
{'upc': '4712052311763', 'name': "Moshi - 5.9' FireWire 800 Cable", 'manufacturer': 'Moshi', 'score': 6.095643}
Moshi - 5.9' FireWire 800 Cable
{'upc': '85387303487', 'name': 'Griffin Technology - USB 2.0 Dock Connector Cable for Apple&#xAE; iPod&#xAE; and iPhone', 'manufacturer': 'Griffin Technology', 'score': 5.8632393}
Griffin Technology - USB 2.0 Dock Connector Cable for Apple&#xAE; iPod&#xAE; and iPhone
{'upc': '50036311939', 'name': 'JBL - On Beat Speaker Dock for Apple&#xAE; iPad&#xAE;, iPhone&#xAE; and iPod&#xAE; - Black', 'manufacturer': 'JBL', 'score': 5.856168}
JBL - On Beat Speaker Dock for Apple&#xAE; iPad&#xAE;, iPhone&#xAE; and iPod&#xAE; - Black


## Create Signals Boosts (Signals Aggregation)

### Listing 4.6

In [5]:
signals_collection = engine.get_collection("signals")

signals_boosting_collection = engine.create_collection("signals_boosting")
print("Aggregating Signals to Create Signals Boosts...")
create_view(signals_collection, "signals")

signals_aggregation_query = """
SELECT q.target AS query, c.target AS doc, COUNT(c.target) AS boost
FROM signals c LEFT JOIN signals q ON c.query_id = q.query_id
WHERE c.type = 'click' AND q.type = 'query'
GROUP BY query, doc
ORDER BY boost DESC
"""

dataframe = spark.sql(signals_aggregation_query)
signals_boosting_collection.write_from_dataframe(dataframe)
print("Signals Aggregation Completed!")

Wiping "signals_boosting" collection
Creating "signals_boosting" collection
Status: Success
Aggregating Signals to Create Signals Boosts...
Signals Aggregation Completed!


## Search with Signals Boosts Applied

### Listing 4.7

In [ ]:
query = "ipad"
signals_boosts_query = {
    "query": query,
    "fields": ["doc", "boost"],
    "limit": 10,
    "params": {
        "defType": "edismax",
        "qf": "query",
        "sort": "boost desc"
    }
}
response = signals_boosting_collection.search(signals_boosts_query)
boosts_docs = engine.docs_from_response(response)
print(f"Boost Documents: \n{boosts_docs}")
boost_query = " ".join([f'"{entry["doc"]}"^{str(entry["boost"])}' 
                           for entry in boosts_docs])
print(f"\nBoost Query: \n{boost_query}")

request = product_search_request(query)
request["params"]["boost"] = "sum(1,query({! df=upc v=$signals_boosting}))"
request["params"]["signals_boosting"] = boost_query
response = products_collection.search(request)
display_product_search(query, engine.docs_from_response(response))

Boost Documents: 
[{'doc': '885909457588', 'boost': 966}, {'doc': '885909457595', 'boost': 205}, {'doc': '885909471812', 'boost': 202}, {'doc': '886111287055', 'boost': 109}, {'doc': '843404073153', 'boost': 73}, {'doc': '635753493559', 'boost': 62}, {'doc': '885909457601', 'boost': 62}, {'doc': '885909472376', 'boost': 61}, {'doc': '610839379408', 'boost': 29}, {'doc': '884962753071', 'boost': 28}]

Boost Query: 
"885909457588"^966 "885909457595"^205 "885909471812"^202 "886111287055"^109 "843404073153"^73 "635753493559"^62 "885909457601"^62 "885909472376"^61 "610839379408"^29 "884962753071"^28
{'query': 'ipad', 'fields': ['upc', 'name', 'manufacturer', 'score'], 'limit': 5, 'params': {'qf': 'name manufacturer longDescription', 'defType': 'edismax', 'indent': 'true', 'sort': 'score desc, upc asc', 'boost': 'sum(1,query({! df=upc v=$signals_boosting}))', 'signals_boosting': '"885909457588"^966 "885909457595"^205 "885909471812"^202 "886111287055"^109 "843404073153"^73 "635753493559"^62 "

## Success!

You have now implemented your first AI-powered search algorithm: Signals Boosting. This is an overly simplistic implementation (we'll dive much deeper into signals boosting improvements in chapter 8), but it demonstrates the power of leveraging reflected intelligence quite well. We will dive into other Reflected Intelligence techniques in future chapters, such as Collaborative Filtering (in chapter 9 - Personalized Search) and Machine-learned Ranking (in chapter 10 - Learning to Rank).

Up next: Chapter 5 - [Knowledge Graph Learning](../ch05/1.open-information-extraction.ipynb)

